In [1]:
import math
import numpy as np
import pandas as pd
import seaborn as sns
import os
import sklearn
from sklearn.metrics import precision_score, recall_score, roc_auc_score, matthews_corrcoef, balanced_accuracy_score, confusion_matrix, f1_score

from imblearn.under_sampling import RandomUnderSampler, NearMiss, TomekLinks, ClusterCentroids
from imblearn.ensemble import EasyEnsembleClassifier, BalancedBaggingClassifier
from imblearn.pipeline import Pipeline
from sklearn.datasets import make_classification
import sys

sys.path.append('/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/')
from RF_atomver import *


In [36]:
df = pd.read_csv('/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/GP/GP_results/GP_test_results_all_NEK.csv')
df['strategy'] = df['strategy'].astype(str)
df

,model,accuracy,precision,recall,specificity,TN,FN,FP,TP,cm,ROC-AUC,MCC,Balanced Accuracy,f1,NEK,strategy,feat_type,model_type
0,NEK2_binding_moe_scaled_GP,0.957597,0.000000,0.000000,1.000000,271,12,0,0,"[271, 0, 12, 0]",0.500000,0.000000,0.500000,0.000000,NEK2_binding,scaled,moe,GP
1,NEK2_binding_moe_UNDER_GP,0.738516,0.102564,0.666667,0.741697,201,4,70,8,"[201, 70, 4, 8]",0.704182,0.184161,0.704182,0.177778,NEK2_binding,UNDER,moe,GP
2,NEK2_binding_moe_SMOTE_GP,0.968198,0.800000,0.333333,0.996310,270,8,1,4,"[270, 1, 8, 4]",0.664822,0.504211,0.664822,0.470588,NEK2_binding,SMOTE,moe,GP
3,NEK2_binding_moe_ADASYN_GP,0.961131,0.571429,0.333333,0.988930,268,8,3,4,"[268, 3, 8, 4]",0.661132,0.418102,0.661132,0.421053,NEK2_binding,ADASYN,moe,GP
4,NEK2_binding_mfp_scaled_GP,0.957597,0.000000,0.000000,1.000000,271,12,0,0,"[271, 0, 12, 0]",0.500000,0.000000,0.500000,0.000000,NEK2_binding,scaled,mfp,GP
5,NEK2_binding_mfp_UNDER_GP,0.954064,0.000000,0.000000,0.996310,270,12,1,0,"[270, 1, 12, 0]",0.498155,-0.012531,0.498155,0.000000,NEK2_binding,UNDER,mfp,GP
6,NEK2_binding_mfp_SMOTE_GP,0.964664,1.000000,0.166667,1.000000,271,10,0,2,"[271, 0, 10, 2]",0.583333,0.400918,0.583333,0.285714,NEK2_binding,SMOTE,mfp,GP
7,NEK2_binding_mfp_ADASYN_GP,0.964664,1.000000,0.166667,1.000000,271,10,0,2,"[271, 0, 10, 2]",0.583333,0.400918,0.583333,0.285714,NEK2_binding,ADASYN,mfp,GP
8,NEK2_inhibition_moe_scaled_GP,0.931540,0.000000,0.000000,1.000000,381,28,0,0,"[381, 0, 28, 0]",0.500000,0.000000,0.500000,0.000000,NEK2_inhibition,scaled,moe,GP
9,NEK2_inhibition_moe_UNDER_GP,0.865526,0.309859,0.785714,0.871391,332,6,49,22,"[332, 49, 6, 22]",0.828553,0.438116,0.828553,0.444444,NEK2_inhibition,UNDER,moe,GP


In [37]:
list1 = list(df.columns)
list2 = ['recall', 'precision']
list3 = [m for m in list1 if m not in list2]
# list3

In [57]:
def rank(df, compare_type, metrics): 
    # metrics = ['recall', 'precision']
    # compare_type = 'NEK'

    if not isinstance(compare_type, list):
        compare_type = [compare_type]
    copydf = df.copy()
    temp_groupby = copydf.groupby(compare_type, group_keys=False)[metrics].agg('max')
    print(temp_groupby)
    grouped = df.groupby(compare_type, group_keys=False).apply(lambda x:x.sort_values(by=metrics, ascending=[False]*len(metrics))) 
        # can do any lsit of metrics 
    cols = list(df.columns) 
    # keep_cols = ['model','model_type','NEK', 'feat_type','cm']
    # keep_cols = ['model','cm']
    # keep_cols.append(compare_type)
    # keep_cols.extend(compare_type)
    # keep_cols.append(metrics) 
    # keep_cols.extend(metrics)
    keep_cols = ['model', 'cm'] + compare_type + metrics
 
    # other_cols = [col for col in cols if col not in keep_cols]
    # print(cols)
    # print(other_cols)
    ranked_df = grouped.reset_index(drop=True)[keep_cols]
    # ranked_df = grouped.reset_index(drop=True).drop(columns=other_cols)

    ranked_df
    return ranked_df

In [58]:
nek2_bind = df[df['NEK'] == 'NEK2_binding']

compare_type =['strategy']
metrics=['recall', 'ROC-AUC', 'f1']

nek2_ranked = rank(nek2_bind, compare_type,metrics)  
nek2_ranked

            recall   ROC-AUC        f1
strategy                              
ADASYN    0.333333  0.661132  0.421053
SMOTE     0.333333  0.664822  0.470588
UNDER     0.666667  0.704182  0.177778
scaled    0.000000  0.500000  0.000000


,model,cm,strategy,recall,ROC-AUC,f1
0,NEK2_binding_moe_scaled_GP,"[271, 0, 12, 0]",scaled,0.000000,0.500000,0.000000
1,NEK2_binding_moe_UNDER_GP,"[201, 70, 4, 8]",UNDER,0.666667,0.704182,0.177778
2,NEK2_binding_moe_SMOTE_GP,"[270, 1, 8, 4]",SMOTE,0.333333,0.664822,0.470588
3,NEK2_binding_moe_ADASYN_GP,"[268, 3, 8, 4]",ADASYN,0.333333,0.661132,0.421053
4,NEK2_binding_mfp_scaled_GP,"[271, 0, 12, 0]",scaled,0.000000,0.500000,0.000000
5,NEK2_binding_mfp_UNDER_GP,"[270, 1, 12, 0]",UNDER,0.000000,0.498155,0.000000
6,NEK2_binding_mfp_SMOTE_GP,"[271, 0, 10, 2]",SMOTE,0.166667,0.583333,0.285714
7,NEK2_binding_mfp_ADASYN_GP,"[271, 0, 10, 2]",ADASYN,0.166667,0.583333,0.285714


In [59]:
print(nek2_bind['strategy'].unique())

['scaled' 'UNDER' 'SMOTE' 'ADASYN']


In [45]:
grouped = nek2_bind.groupby('strategy')
for name, group in grouped:
    print(f"Group: {name}")
    # print(group)

Group: ADASYN
Group: SMOTE
Group: UNDER
Group: scaled
